# Create Disk Snapshot & Forensic Disk (ipynb)

## Install Dependencies

Install the dependencies `ipywidgets` and `pandas`. Skip the next cell if they had already been installed.

In [ ]:
!pip3 install ipywidgets pandas

## Imports and Configuration

In [ ]:
import ipywidgets as widgets
import json
import os
import pandas as pd

from IPython.display import HTML, display

# extend width of widgets
display(HTML('''<style>
    .widget-label { min-width: 24ex !important; font-weight:bold; }
</style>'''))

# extend width and max rows of pandas output
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
# [OPTIONAL] authenticate using your service account
!gcloud auth activate-service-account --key-file <json_key_file>

## Steps

**Specify the following information**
| Fields      | Description |
| ----------- | ----------- |
| `Destination Project`   | Project id of analyst project    |
| `Destination Zone`   | Zone of compute instance the created forensic disk would be attached to, e.g. `asia-southeast1-b`    |
| `Target Instance`   | Name of compute instance the created forensic disk would be attached to, e.g. `sift-instance`    |
| `Source Project`   | Project id of target project (that contains potentially compromised compute instance    |

In [ ]:
# create text boxes for user input
dst_project = widgets.Text(description = "Destination Project: ", disabled=False, layout=widgets.Layout(width='40%'))
dst_zone = widgets.Text(description = "Destination Zone: ", disabled=False, layout=widgets.Layout(width='40%'))
target_instance = widgets.Text(description = "Target Instance: ", disabled=False, layout=widgets.Layout(width='40%'))
src_project = widgets.Text(description = "Source Project: ", disabled=False, layout=widgets.Layout(width='40%'))

display(dst_project, dst_zone, target_instance, src_project)

In [ ]:
# store user input in environment variables for use in subsequent comamnds
os.environ['DST_PROJECT'] = dst_project.value
os.environ['DST_ZONE'] = dst_zone.value
os.environ['TARGET_INSTANCE'] = target_instance.value
os.environ['SRC_PROJECT'] = src_project.value

In [ ]:
# list disks in target (src) project
!gcloud compute disks list --project $SRC_PROJECT

**Specify the following information from the above output**
| Fields      | Description |
| ----------- | ----------- |
| `Source Disk`   | Name of the disk in target (src) project to create a snapshot of    |
| `Source Zone`   | Zone of the disk in target (src) project to create a snapshot of, e.g. `asia-southeast1-b`    |
| `Snapshot Name`   | Name of created snapshot of your choice    |
| `Destination Disk`   | Name of created forensics disk of your choice    |

In [ ]:
# create text boxes for user input
src_disk = widgets.Text(description = "Source Disk: ", disabled=False, layout=widgets.Layout(width='40%'))
src_zone = widgets.Text(description = "Source Zone: ", disabled=False, layout=widgets.Layout(width='40%'))
ss_name = widgets.Text(description = "Snapshot Name: ", disabled=False, layout=widgets.Layout(width='40%'))
dst_disk = widgets.Text(description = "Destination Disk: ", disabled=False, layout=widgets.Layout(width='40%'))
display(src_disk, src_zone, ss_name, dst_disk)

In [ ]:
# store user input in environment variables for use in subsequent comamnds
os.environ['SRC_DISK'] = src_disk.value
os.environ['SRC_ZONE'] = src_zone.value
os.environ['SS_NAME'] = ss_name.value
os.environ['DST_DISK'] = dst_disk.value

In [ ]:
# create snapshot SS_NAME from SRC_DISK
!gcloud compute disks snapshot $SRC_DISK --project $SRC_PROJECT --zone $SRC_ZONE --snapshot-names $SS_NAME

In [ ]:
# create DST_DISK from snapshot SS_NAME
!gcloud compute disks create $DST_DISK --project $DST_PROJECT --zone $DST_ZONE --source-snapshot projects/$SRC_PROJECT/global/snapshots/$SS_NAME

In [ ]:
# attach DST_DISK to TARGET_INSTANCE
!gcloud compute instances attach-disk $TARGET_INSTANCE --project $DST_PROJECT --zone $DST_ZONE --disk $DST_DISK --mode ro 

In [ ]:
# delete snapshot SS_NAME from SRC_PROJECT
!gcloud compute snapshots delete $SS_NAME --project $SRC_PROJECT --quiet

To mount in the destination instance, the following commands can be used:
```
# create mount point
sudo mkdir <mnt_pt>

# identify device id of attached disk
lsblk

# mount the attached disk:
sudo mount -o ro,noload /dev/<device_id> <mnt_pt>
```

To unmount the attached disk:
```
sudo umount <mnt_pt>
```